In [126]:
import pandas as pd
import numpy as np

In [127]:
# read in final_df cleaned file
data_df = pd.read_csv('final_df_zip.csv')

In [128]:
# read in unemployment
unemployment = pd.read_csv('unemployment.csv')

In [129]:
# read in income
income = pd.read_csv('2023-personal-income-tax-statistics-by-zipcode.csv')

In [131]:
# lag income by year bc not reported til following year avoid data leackage
income['Taxable Year'] = income['Taxable Year'] + 1

In [133]:
income['City'] = income['City'].apply(lambda x: x.upper())
income['County'] = income['County'].apply(lambda x: x)

In [134]:
income = income.loc[income['City'] != 'CITY UNMATCHED']

In [135]:
income['Zip Code'] = income['Zip Code'].astype(int)

In [136]:
# Rename column 'target' to 'new_target'
income.rename(columns={'Taxable Year': 'Year'}, inplace=True)

In [138]:
# Rename column 'target' to 'new_target'
income.rename(columns={'Zip Code': 'zip_code'}, inplace=True)

In [139]:
data_df.dropna(inplace=True)
data_df['Month'] = data_df['sale_datetime'].apply(lambda x: x.split('-')[1])
data_df['Year'] = data_df['sale_datetime'].apply(lambda x: int(x.split('-')[0]))

In [ ]:
data_df.loc[data_df['Year'] < data_df['building_year_built']]

In [141]:
# merge datasets
data_df = pd.merge(data_df, income[['zip_code','Year', 'Geo ZipCode','Geo City', 'Returns',
       'California AGI', 'Total Tax Liability']], on=['zip_code', 'Year'], how='left')

In [142]:
data_df.dropna(inplace=True)

In [ ]:
data_df

In [145]:
# function to get geo location from column

def strip_geo(coord):
    
    coord = coord.split('POINT')[-1]
    
    lon = coord.split('(')[-1].split(' ')[0].strip()
    
    lat = coord.split(')')[0].split(' ')[-1].strip()
    
    return f'{lon},{lat}'

In [146]:
# apply funtion
data_df['City_Geo'] = data_df['Geo City'].apply(strip_geo)

In [ ]:
data_df.head()

In [148]:

geo_city = data_df['City_Geo'].unique()

In [ ]:
geo_city[0]

In [59]:
# use api to do reverse lookup

import requests
def get_city(lon, lat):
    url = f"https://nominatim.openstreetmap.org/reverse?format=json&lat={lat}&lon={lon}"
    response = requests.get(url, headers={'User-Agent': 'your-app-name'})
    data = response.json()
    
    # Try different possible keys for city name
    address = data['address']
    city_keys = ['city', 'town', 'village', 'municipality', 'suburb']
    
    for key in city_keys:
        if key in address:
            return address[key]
    
    # If no city-like key is found, return the most specific place name available
    return address.get('display_name', 'Location not found')

In [150]:
# get city names from geolocation and add to df

city_dict = {}

for city in geo_city:
    
    lon = city.split(',')[0]
    
    lat = city.split(',')[-1]
    
    city_name = get_city(lon, lat)
    
    city_dict[city] = city_name.upper()
    
data_df['City_Name'] = data_df['City_Geo'].apply(lambda x: city_dict[x])

In [ ]:
data_df.head()

In [ ]:
data

In [27]:
data_df.dropna(inplace=True)

In [ ]:
unemployment

In [156]:
unemployment['County_check'] = unemployment['Area'].apply(lambda x: x.split(' ')[-1])

In [ ]:
unemployment = unemployment.loc[unemployment['County_check'] != 'County']
unemployment

In [ ]:
unemployment['County_check'].unique()

In [159]:
filtered_df = unemployment.loc[unemployment['County_check'].isin(['town', 'City', 'city'])]

In [ ]:
filtered_df

In [ ]:
filtered_df['City'] = filtered_df['Area'].apply(lambda x: x.split('city')[0].upper().strip())

In [ ]:
filtered_df

In [163]:
month_to_year = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}


In [ ]:
filtered_df = filtered_df.loc[filtered_df['Period'] != 'Annual']
filtered_df['Month'] = filtered_df['Period'].apply(lambda x: month_to_year[x])

In [ ]:
filtered_df['Month'] = filtered_df['Period'].apply(lambda x: month_to_year[x])

In [ ]:
filtered_df['Month'] = filtered_df['Month'] + 1

In [167]:
filtered_df = filtered_df[['City', 'Year', 'Month', 'Unemployment Rate', 'Labor Force', 'Employment']]

In [ ]:
filtered_df

In [ ]:
filtered_df['City'].unique()

In [170]:
# Rename column 'target' to 'new_target'
filtered_df.rename(columns={'City': 'City_Name'}, inplace=True)

In [174]:
data_df['Month'] = data_df['Month'].astype(int)

In [ ]:
# merge uneployment on city with main df

city_df = pd.merge(data_df, filtered_df, on=['Year', 'Month', 'City_Name'])
len(city_df)

In [ ]:
city_df['City_Name'].unique()

In [177]:
# export to csv to use for lags

city_df.to_csv('city_df_2.csv')

In [ ]:
un_year_df = pd.read_csv('DA20251525.txt')
un_year_df

In [ ]:
un_year_df.dtypes

In [ ]:
un_year_df = un_year_df.loc[un_year_df['Period'] != 'Annual']
un_year_df['Month'] = un_year_df['Period'].apply(lambda x: month_to_year[x])
un_year_df['County'] = un_year_df['Area'].apply(lambda x: x.split('County')[0].strip())

un_year_df = un_year_df[['Year', 'Month', 'Unemployment Rate', 'Labor Force', 'Employment', 'County']]

In [ ]:
un_year_df

In [ ]:
un_year_df['County'].unique()

In [ ]:
data_df['County'].unique()

In [ ]:
county_df = pd.merge(data_df, un_year_df, on=['Year', 'Month', 'County'])
len(county_df)

In [152]:
county_df.to_csv('county_df.csv')